In [8]:
import pandas as pd

In [9]:
job_info_with_trace_with_delay_with_host = pd.read_csv('./job_info_with_trace_with_delay_with_host.csv')

c:\Users\MichalFaciszewski\Documents\mgr_study\MGR\.venv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (124,162,171,261,355,356,358,362,364,372,376,377,378,380) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
walltime = job_info_with_trace_with_delay_with_host['walltime']
job_info_with_trace_with_delay_with_host_without_walltime = job_info_with_trace_with_delay_with_host.drop('walltime', axis=1)

In [11]:
job_info_with_trace_with_delay_with_host_without_walltime['text repr'] = job_info_with_trace_with_delay_with_host_without_walltime.apply(lambda row: ';'.join(map(str, row)), axis=1)
job_info_with_trace_with_delay_with_host_without_walltime['text repr']

0        0;10.109.0.100;2807428285.0;t;t;DONE;167703062...
1        1;10.109.0.111;2805880462.0;t;t;DONE;167680817...
2        2;10.109.0.112;2805777586.0;t;t;DONE;167680779...
3        3;10.109.0.112;2805896694.0;t;t;DONE;167680825...
4        4;10.109.0.113;2805802601.0;t;t;DONE;167680788...
                               ...                        
10095    10095;wp-10-23.lnl.infn.it;2807415318.0;t;t;DO...
10096    10096;wp-10-24.lnl.infn.it;2807268648.0;t;t;DO...
10097    10097;wp-10-24.lnl.infn.it;2807268668.0;t;t;DO...
10098    10098;wp-10-24.lnl.infn.it;2807268643.0;t;t;DO...
10099    10099;wp-10-24.lnl.infn.it;2805895019.0;t;t;DO...
Name: text repr, Length: 10100, dtype: object

In [22]:
job_descriptions = job_info_with_trace_with_delay_with_host_without_walltime['text repr'].values

In [25]:
def get_training_corpus(dataset):
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]

In [30]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.BPE())
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])

In [32]:
tokenizer.train_from_iterator(get_training_corpus(job_descriptions), trainer=trainer)


In [ ]:
# cls_token_id = tokenizer.token_to_id("[CLS]")
# sep_token_id = tokenizer.token_to_id("[SEP]")
# print(cls_token_id, sep_token_id)

2 3


In [33]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)


In [36]:
tokenizer.decoder = decoders.ByteLevel()
tokenizer.save("tokenizer.json")
# load tokenizer from file
# new_tokenizer = Tokenizer.from_file("tokenizer.json")


In [39]:
encoding = tokenizer.encode(job_descriptions[0])
print(encoding.tokens)
print(encoding.type_ids)


['0', ';', '10', '.', '109', '.', '0', '.', '100', ';', '28074282', '85', '.', '0', ';', 't', ';', 't', ';', 'done', ';', '1677030629', '.', '0', ';', '2023', '-', '02', '-', '26', 'Ġ18', ':', '16', ':', '51', '.', '660', '442', ';', '98', '.', '91', ';', '715', '27', '.', '19', ';', '10', '.', '109', '.', '0', '.', '100', ';', '1427456', '.', '0', ';', '98', '.', '91', '%;', '1677427', '927000', '.', '0', ';', '2807256851', '.', '0', ';', '28074282', '85', '.', '0', ';', '1', '.', '0', ';', '72000', '.', '0', ';', '16773556', '81000', '.', '0', ';', '16774', '279', '25000', '.', '0', ';', '16773556', '80', '999', '.', '0', ';', '1427456', '.', '0', ';', 'alice', '::', 'uib', '::', 'slurm', ';', '1677428', '217000', '.', '0', ';', '27408', '.', '0', ';{', 'ĠĠ', 'Ġ"', 'user', '":"', 'aliprod', '",', 'ĠĠ', 'Ġ"', 'jobtag', '":[', 'ĠĠĠĠ', 'Ġ"', 'comment', ':', 'pp', ',', 'Ġ13', 'Ġtev', 'Ġ-', 'Ġpythia', '-', '8', 'Ġgeant', '4', 'Ġextra', '"', 'ĠĠ', 'Ġ],', 'ĠĠ', 'Ġ"', 'packages', '":[', 'ĠĠĠ

In [40]:
tokenizer.decode(encoding.ids)


'0;10.109.0.100;2807428285.0;t;t;done;1677030629.0;2023-02-26 18:16:51.660442;98.91;71527.19;10.109.0.100;1427456.0;98.91%;1677427927000.0;2807256851.0;2807428285.0;1.0;72000.0;1677355681000.0;1677427925000.0;1677355680999.0;1427456.0;alice::uib::slurm;1677428217000.0;27408.0;{   "user":"aliprod",   "jobtag":[     "comment:pp, 13 tev - pythia-8 geant4 extra"   ],   "packages":[     "vo_alice@alidpg::prod-202203-02-1",     "vo_alice@aliphysics::v5-09-56l-01-1",     "vo_alice@jemalloc::latest"   ],   "executable":"\\/alice\\/cern.ch\\/user\\/a\\/aliprod\\/bin\\/aliroot_dpgsim.sh",   "arguments":" --run 263654 --mode full --uid 75704 --nevents 500 --generator pwglf:pythia8_monash2013_str002 --geant4 --its-material kspdsichip=1.3,kspdsisens=1.3,kspdalbus=1.3,kspdcoolpipes=1.3,ksddsiall=1.12",   "inputfile":[     "lf:\\/alice\\/sim\\/2021\\/lhc21l6\\/ocdb\\/263654\\/ocdbsim.root",     "lf:\\/alice\\/sim\\/2021\\/lhc21l6\\/ocdb\\/263654\\/ocdbrec.root"   ],   "jdlpath":"\\/alice\\/cern.ch\\/